Load_helper_only to drop_table

In [0]:
%run /Repos/a.torralbo@ucl.ac.uk/ccu013_02/atlas/00_Load_helpers

1) Identify broad covid cohorts
2) most severe (max cohort)
3) date for most severe
4) first events in most severe 
5) before 23 Jan 2021

explore tables:

In [0]:
%sql
select count(distinct(person_id_deid)) from dars_nic_391419_j3w9t_collab.ccu013_phe_cohort

In [0]:
%sql
select * from dars_nic_391419_j3w9t_collab.ccu013_phe_cohort limit 3;

In [0]:
%sql
select * from dars_nic_391419_j3w9t_collab.ccu013_covid_trajectory_paper_cohort limit 3;

In [0]:
%sql
select distinct(covid_phenotype) from dars_nic_391419_j3w9t_collab.ccu013_covid_trajectory_paper_cohort;

In [0]:
traj = spark.sql("""
select * 
from dars_nic_391419_j3w9t_collab.ccu013_covid_trajectory_paper_cohort limit 10 """)

traj.schema

1) broad cohort and first date pher phenotype within broad group

In [0]:
%sql 
select p.person_id_deid,
       p.covid_phenotype, 
       min(p.date) as covid_phenotype_first_event,
CASE 
  WHEN p.covid_phenotype LIKE '%01_%' THEN '01_phenotype'
  WHEN p.covid_phenotype LIKE '%02_%' THEN '02_phenotype'
  WHEN p.covid_phenotype LIKE '%03_%' THEN '03_phenotype'
  WHEN p.covid_phenotype LIKE '%04_%' THEN '04_phenotype'
  ELSE 'unknown'
END AS covid_phenotype_broad
from dars_nic_391419_j3w9t_collab.ccu013_covid_trajectory_paper_cohort p
group by p.person_id_deid, p.covid_phenotype;

In [0]:
ccu013_covid_first_event_per_phenotype = spark.sql(f'''
select p.person_id_deid,
       p.covid_phenotype, 
       min(p.date) as covid_phenotype_first_event,
CASE 
  WHEN p.covid_phenotype LIKE '%01_%' THEN '01_phenotype'
  WHEN p.covid_phenotype LIKE '%02_%' THEN '02_phenotype'
  WHEN p.covid_phenotype LIKE '%03_%' THEN '03_phenotype'
  WHEN p.covid_phenotype LIKE '%04_%' THEN '04_phenotype'
  ELSE 'unknown'
END AS covid_phenotype_broad
from dars_nic_391419_j3w9t_collab.ccu013_covid_trajectory_paper_cohort p
group by p.person_id_deid, p.covid_phenotype''')

In [0]:
ccu013_covid_first_event_per_phenotype.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_first_event_per_phenotype")

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_first_event_per_phenotype limit 3;

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_first_event_per_phenotype where person_id_deid == '';

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_first_event_per_phenotype where person_id_deid == '';

2) select most severe phenotype from broad groups

In [0]:
%sql 
select s.person_id_deid, 
       max(s.covid_phenotype_broad) as most_severe_phenotype_broad
from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_first_event_per_phenotype s
group by s.person_id_deid;   

In [0]:
ccu013_covid_most_severe_phenotype = spark.sql(f'''
select s.person_id_deid, 
       max(s.covid_phenotype_broad) as most_severe_phenotype_broad
from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_first_event_per_phenotype s
group by s.person_id_deid''')

In [0]:
ccu013_covid_most_severe_phenotype.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_most_severe_phenotype")

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_most_severe_phenotype limit 4;

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_most_severe_phenotype where person_id_deid == '';

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_most_severe_phenotype;

3) select date and phenotype for most severe - join with first event per phenotype (from 1)
- if multiple phenotypes within the same broad groups there might be multiple entries here 

In [0]:
%sql 
select *  
from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_first_event_per_phenotype limit 3;

In [0]:
%sql 
select t.person_id_deid, 
       t.covid_phenotype_first_event as earliest_most_severe,
       t.covid_phenotype, 
       t.covid_phenotype_broad as most_severe  
from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_first_event_per_phenotype t
inner join dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_most_severe_phenotype c
on t.person_id_deid = c.person_id_deid
where t.covid_phenotype_broad = c.most_severe_phenotype_broad
group by t.person_id_deid, t.covid_phenotype_broad, t.covid_phenotype, t.covid_phenotype_first_event;

In [0]:
ccu013_covid_event_most_severe_all_1d = spark.sql(f'''
select t.person_id_deid, 
       t.covid_phenotype_first_event as earliest_most_severe,
       t.covid_phenotype, 
       t.covid_phenotype_broad as most_severe
from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_first_event_per_phenotype t
inner join dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_covid_most_severe_phenotype c
on t.person_id_deid = c.person_id_deid
where t.covid_phenotype_broad = c.most_severe_phenotype_broad
group by t.person_id_deid, t.covid_phenotype_broad, t.covid_phenotype, t.covid_phenotype_first_event''')

In [0]:
ccu013_covid_event_most_severe_all_1d.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_event_1d")

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_event_1d limit 4;

In [0]:
%sql 
--multiple entries if multiple pheno within broad group 
select count(*) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_event_1d; 

4) get first one in broad severity group - gives 1 per indiv - total N as in traject ccu013 table 

In [0]:
%sql 
select s.person_id_deid, 
       s.most_severe,
       min(s.earliest_most_severe), 
CASE
    WHEN s.most_severe LIKE '%01_%' THEN 'cohort_01' 
    WHEN s.most_severe LIKE '%02_%' THEN 'cohort_02'
    WHEN s.most_severe LIKE '%03_%' THEN 'cohort_03'
    WHEN s.most_severe LIKE '%04_%' THEN 'cohort_04' 
    ELSE 'unknown'
    END 
    AS covid_cohort
from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_event_1d s
group by s.person_id_deid, s.most_severe;

In [0]:
df_most_severe_covid_first_event = spark.sql(f'''
select s.person_id_deid, 
       s.most_severe,
       min(s.earliest_most_severe) as earliest_most_severe, 
CASE
    WHEN s.most_severe LIKE '%01_%' THEN 'cohort_01' 
    WHEN s.most_severe LIKE '%02_%' THEN 'cohort_02'
    WHEN s.most_severe LIKE '%03_%' THEN 'cohort_03'
    WHEN s.most_severe LIKE '%04_%' THEN 'cohort_04' 
    ELSE 'unknown'
    END 
    AS covid_cohort
from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_event_1d s
group by s.person_id_deid, s.most_severe''')

In [0]:
df_most_severe_covid_first_event.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest")

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest;

5) update first events before 23 Jan 2021

In [0]:

df_most_severe_covid_first_event = spark.sql(f'''
select s.person_id_deid, 
       s.most_severe,
       min(s.earliest_most_severe) as earliest_most_severe, 
CASE
    WHEN s.most_severe LIKE '%01_%' THEN 'cohort_01' 
    WHEN s.most_severe LIKE '%02_%' THEN 'cohort_02'
    WHEN s.most_severe LIKE '%03_%' THEN 'cohort_03'
    WHEN s.most_severe LIKE '%04_%' THEN 'cohort_04' 
    ELSE 'unknown'
    END 
    AS covid_cohort       
from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_event_1d s
group by s.person_id_deid, s.most_severe
having min(s.earliest_most_severe) <= to_date('2021-01-23')''')

In [0]:
df_most_severe_covid_first_event.write.mode("overwrite").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest")

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest;

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest limit 3;

In [0]:
%sql
select count(distinct(person_id_deid)) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest where covid_cohort LIKE '%_01%';

In [0]:
%sql
select count(distinct(person_id_deid)) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest where covid_cohort LIKE '%_02%';

In [0]:
%sql
select count(distinct(person_id_deid)) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest where covid_cohort LIKE '%_03%';

In [0]:
%sql
select count(distinct(person_id_deid)) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest where covid_cohort LIKE '%_04%';

In [0]:

%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest where person_id_deid = '';

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest limit 3;

In [0]:
%sql 
select MAX(earliest_most_severe) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest;

In [0]:

%sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_covid_trajectory_paper_cohort where person_id_deid = '';

In [0]:
%sql 
select MAX(earliest_most_severe) from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest;

test unique id per severity cohort (done in Rstudio)

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_phe_cohort_most_severe_covid_cohort_earliest where person_id_deid = '';

In [0]:
%sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_covid_trajectory_paper_cohort where person_id_deid = '';